In [ ]:
from fastapi import FastAPI, Request
import requests
import os
from dotenv import load_dotenv

load_dotenv()

GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [ ]:
REVIEW_PROMPT_TEMPLATE_V1 = """
You are a senior software engineer reviewing a GitHub Pull Request.

Please analyze the following code diff and give feedback in the following structure:

1. **Key Changes**: Summarize what changed.
2. **Code Clarity**: Is the code readable? What could be improved?
3. **Comments and Naming**: Are function and variable names clear and descriptive? (For Go, refer to the Google Go Style Guide: https://google.github.io/styleguide/go/)
4. **Complexity Reduction**: Are there complex logic or structures that could be simplified?
5. **Bug Risks**: Any potential bugs or edge cases?
6. **Best Practices**: Are best practices followed? Suggest improvements with code.
7. **Allow MR to Merge?**: Should this be merged? Answer: Yes / Yes with minor changes / No. Explain why.

Code Diff:
---
{diff}
---
If the provided code is in Go, pay special attention to adherence to the Google Go Style Guide (https://google.github.io/styleguide/go/) in relevant sections, particularly Naming Conventions and Best Practices.
Respond only with a detailed review following the 7 steps above. Be specific, constructive, and provide examples from the code where appropriate to illustrate your points.
Note: Try to write the best possible summary
"""

REVIEW_PROMPT_TEMPLATE_V2 = """
You are a highly experienced Senior Software Engineer tasked with conducting a thorough Git Merge Request (MR) review. Your goal is to provide constructive, actionable feedback that helps improve code quality, maintainability, and robustness.
Please meticulously review the following code changes. Analyze them and provide feedback structured as follows. If the provided code is in Go, pay special attention to adherence to the Google Go Style Guide (https://google.github.io/styleguide/go/) in relevant sections, particularly Naming Conventions and Best Practices.
1. Overall Summary & Key Changes:
    * Briefly state the main purpose or goal of this MR (e.g., "This MR introduces a new feature for X," or "This MR refactors Y to improve performance/clarity").
    * Summarize the most significant additions, modifications, or removals in terms of modules, functions, or core logic.
2. Code Clarity & Readability:
    * Is the code's intent immediately obvious? Is the logical flow easy to follow?
    * Are there any sections that are overly dense, convoluted, or require significant mental effort to decipher?
    * Suggest specific improvements for clarity (e.g., breaking down complex functions into smaller, single-responsibility units, simplifying conditional expressions, improving overall code structure or layout).
    * Are variable and function scopes as limited as possible?
3. Naming Conventions & Comments:
    * Are function, variable, class, struct, interface, and module names clear, descriptive, unambiguous, and consistent with common conventions for the language/project? (For Go, refer to the Google Go Style Guide: https://google.github.io/styleguide/go/)
    * Are comments used effectively to explain why something is done (the intent, trade-offs, or complex logic), rather than just what is being done (which the code should ideally convey)?
    * Are there any misleading, outdated, or redundant comments? Are there places where comments are missing and would be beneficial (e.g., for non-obvious business logic or complex algorithms)?
    * Is package naming and organization logical (if applicable)?
4. Simplicity & Complexity Reduction (KISS Principle):
    * Are there any overly complex conditional statements (e.g., deeply nested if/else), convoluted loops, or unnecessarily complicated algorithms?
    * Can any part of the code be refactored for greater simplicity or to reduce cognitive load without sacrificing functionality or clarity? Provide specific suggestions (e.g., using helper functions, better data structures, or design patterns).
    * Is there any "magic" (unexplained constants, arbitrary-seeming decisions) that could be clarified or made more explicit?
5. Potential Bugs & Edge Cases:
    * Do you foresee any potential bugs, race conditions, resource leaks (e.g., unclosed files/connections, goroutine leaks in Go), or off-by-one errors?
    * Are edge cases (e.g., null/nil inputs, empty collections, zero values, invalid data formats) and error conditions handled gracefully, comprehensively, and explicitly? Are errors propagated or handled appropriately?
    * (For Go: Are error values consistently checked and handled? Is panic used appropriately, i.e., rarely and for truly unrecoverable situations?)
    * Are there any unsafe operations or potential security vulnerabilities (e.g., SQL injection, XSS, improper input validation, data exposure, insecure use of cryptography)?
6. Best Practices & Design Principles:
    * Does the code adhere to relevant language-specific idioms and best practices? (For Go, this includes aspects like effective error handling, interface usage, concurrency patterns, and package design from the Google Go Style Guide.)
    * Does it follow established software design principles (e.g., DRY - Don't Repeat Yourself, SOLID, KISS - Keep It Simple, Stupid)?
    * Are there considerations for performance, scalability, or resource consumption that should be addressed? (e.g., inefficient algorithms, unnecessary allocations, blocking I/O in critical paths).
    * Is the code testable? Are dependencies managed in a way that facilitates testing (e.g., dependency injection)?
    * Are there any anti-patterns present? If so, suggest alternatives.
    * Is logging sufficient and appropriately leveled?
7. Testing (If Applicable/Inferable):
    * If tests are included or modified, are they clear, comprehensive, and effective in verifying the changes?
    * Do tests cover success paths, failure paths, and relevant edge cases for the new/modified code?
    * Are there any obvious missing test cases for the changes introduced?
    * Are tests isolated and repeatable? Do they clean up after themselves?
8. Recommendation & Rationale:
    * Based on your review, provide a clear recommendation:
        * Approve: All looks good, or only trivial nits that can be addressed post-merge or are stylistic preferences not violating guidelines.
        * Approve with suggestions: Minor changes are recommended (e.g., for clarity, minor refactoring) but are not blockers to merging. The author can consider them.
        * Request changes: Significant issues (e.g., potential bugs, major design flaws, missing critical tests, violations of core principles) need addressing before merging.
    * Clearly explain your reasoning, prioritizing critical issues over minor suggestions. If requesting changes, list the specific blocking issues that must be resolved.


Code Diff:
---
{diff}
---
Respond only with a detailed review following the 8 steps above. Be specific, constructive, and provide examples from the code where appropriate to illustrate your points.
"""

In [ ]:
diff_url = "https://patch-diff.githubusercontent.com/raw/baby-hero/go-experiments/pull/1.diff"
headers = {"Authorization": f"token {GITHUB_TOKEN}"}
diff = requests.get(diff_url, headers=headers).text
diff

In [ ]:
diff_url = "https://github.com/baby-hero/go-experiments/pull/2.diff"
headers = {"Authorization": f"token {GITHUB_TOKEN}"}
diff = requests.get(diff_url, headers=headers).text
print(diff)

In [ ]:
prompt_v1 = REVIEW_PROMPT_TEMPLATE_V1.format(diff=diff[:10000])  # truncate if too long
prompt_v2 = REVIEW_PROMPT_TEMPLATE_V2.format(diff=diff[:10000])  # truncate if too long


In [ ]:
from google import genai

client = genai.Client(api_key=GEMINI_API_KEY)

response = client.models.generate_content(
    model="gemini-2.0-flash", contents=prompt_v1
)
print(response.text)

In [ ]:
import google as genai

genai.configure(api_key=GEMINI_API_KEY)

model = genai.GenerativeModel("gemini-2.0-flash")
chat = model.start_chat()

response_v1 = chat.send_message(prompt_v1)
response_v2 = chat.send_message(prompt_v2)
# response_v3 = chat.send_message(prompt_v3)


In [ ]:

print(response_v1.text)


In [ ]:
print(response_v2.text)


In [ ]:
from ai_tools.gemini import get_mr_review
res = get_mr_review(prompt_v1)
print(res)

In [ ]:
owner = "baby-hero"
repo_name = "go-experiments"
pr_number = "3"

headers = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json"
}

comment_url = f"https://api.github.com/repos/{owner}/{repo_name}/issues/{pr_number}/comments"
requests.post(comment_url,
    headers=headers,
    json={"body": res})

In [ ]:
from utils import github_util

result = github_util.post_comment_to_pr(owner, repo_name, pr_number, "Have a nice day!")
print(result.text)